In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python -m pip install --upgrade pip

!pip install tensorflow==2.3.1 

!pip install tensorboard==2.4.1
!pip install torch  #YOLOv5 runs on top of PyTorch, so we need to import it to the notebook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.3.1
Looking in index

In [3]:
import torch # YOLOv5 implemented using pytorch

In [4]:
from IPython.display import Image #this is to render predictions

In [5]:
%cd /content/drive/MyDrive/masked_unmasked_yolov5

/content/drive/MyDrive/masked_unmasked_yolov5


In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [6]:
%cd /content/drive/MyDrive/masked_unmasked_yolov5/yolov5

/content/drive/MyDrive/masked_unmasked_yolov5/yolov5


In [7]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


In [ ]:
import os
from random import choice
import shutil

#arrays to store file names
imgs =[]
xmls =[]

#setup dir names
trainPath = '/content/drive/MyDrive/masked_unmasked_yolov5/data/images/train/'
valPath = '/content/drive/MyDrive/masked_unmasked_yolov5/data/images/val/'
crsPath = '/content/drive/MyDrive/masked_unmasked_yolov5/raw_data' #dir where images and annotations stored

#setup ratio (val ratio = rest of the files in origin dir after splitting into train and test)
train_ratio = 0.8
val_ratio = 0.2


#total count of imgs
totalImgCount = len(os.listdir(crsPath))/2

#soring files to corresponding arrays
for (dirname, dirs, files) in os.walk(crsPath):
    for filename in files:
        if filename.endswith('.txt'):
            xmls.append(filename)
        else:
            imgs.append(filename)


#counting range for cycles
countForTrain = int(len(imgs)*train_ratio)
countForVal = int(len(imgs)*val_ratio)
print("training images are  : ",countForTrain)
print("Validation images are: ",countForVal)

In [8]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
trainImagePath = '/content/drive/MyDrive/masked_unmasked_yolov5/data/images/train/'
trainLabelPath = '/content/drive/MyDrive/masked_unmasked_yolov5/data/labels/train/'
valImagePath = '/content/drive/MyDrive/masked_unmasked_yolov5/data/images/val/'
valLabelPath = '/content/drive/MyDrive/masked_unmasked_yolov5/data/labels/val/'
testPath = '/content/drive/MyDrive/masked_unmasked_yolov5/test'

In [ ]:
def train_test_split(path,neg_path=None, split=0.2):
  print("-------- PROCESS STARTED ---------")

  files = list(set([name[:-4] for name in os.listdir(path)]))   ## Removing duplicate names data...

  print(f">>> This folder has a total number of {len(files)} images <<<")
  random.seed(42)
  random.shuffle(files)

  test_size = int(len(files) * split)
  train_size = len(files) - test_size

  ## Createing requird directories

  os.makedirs(trainImagePath, exist_ok= True)
  os.makedirs(trainLabelPath, exist_ok= True)
  os.makedirs(valImagePath, exist_ok= True)
  os.makedirs(valLabelPath, exist_ok= True)

  ## Copying images to train folder
  
  for filex in tqdm(files[:train_size]):
    if filex == 'classes':
      continue
    shutil.copy2(path + filex + '.jpg', f"{trainImagePath}/" + filex + '.jpg')
    shutil.copy2(path + filex + '.txt', f"{trainLabelPath}/" + filex + '.txt')

  print(f">>>> Training data created with 80% split {len(files[:train_size])} images <<<<")

  if neg_path:
    neg_images = list(set([name[:4] for name in os.listdir(neg_path)]))   ## Removing duplicate names data
    
    for filex in tqdm(neg_images):
      shutil.copy2(neg_path + filex + '.jpg', f"{trainImagePath}/" + filex + '.jpg')
    
    print(f">>>> Total {len(neg_images)} negative images added to the training data <<<<")
    print(f">>>> Total training data created with {len(files[:train_size]) + len(neg_images)} images <<<<")


  ## Copying images to validation folder

  for filex in tqdm(files[train_size:]):
    if filex == 'classes':
       continue
    
    shutil.copy2(path + filex + '.jpg', f"{valImagePath}/" + filex + '.jpg')
    shutil.copy2(path + filex + '.txt', f"{valLabelPath}/" + filex + '.txt')

  print(f">>>> Testing data created with 80% split {len(files[train_size:])} images <<<<")

  print(">>>> TASK COMPLETE <<<<")


### for label_tag
train_test_split('/content/drive/MyDrive/masked_unmasked_yolov5/raw_data/')    ## Without negative images

In [ ]:
!python train.py --img 416 --batch 16 --epochs 500 --data '/content/drive/MyDrive/masked_unmasked_yolov5/data/data.yaml' --weights yolov5s.pt --cache

2023-04-08 19:26:05.326967: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 19:26:06.492454: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/masked_unmasked_yolov5/data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [11]:
#Image(filename= '/content/drive/MyDrive/yolo/unseen_test/')

%cd '/content/drive/MyDrive/masked_unmasked_yolov5/yolov5/'

/content/drive/MyDrive/masked_unmasked_yolov5/yolov5


In [13]:
!python detect.py --source '/content/drive/MyDrive/masked_unmasked_yolov5/test/' --weights '/content/drive/MyDrive/masked_unmasked_yolov5/yolov5/runs/train/exp2/weights/best.pt' --conf 0.1

detect: weights=['/content/drive/MyDrive/masked_unmasked_yolov5/yolov5/runs/train/exp2/weights/best.pt'], source=/content/drive/MyDrive/masked_unmasked_yolov5/test/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-135-g9dd0ad3 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/12 /content/drive/MyDrive/masked_unmasked_yolov5/test/000_1ov3n5_0_jpeg.rf.a23f1c89491779996f4519858277a4e0.jpg: 640x640 2 masks, 11.6ms
image 2/12 /content/drive/MyDrive/masked_unmasked_yolov5/test/000b7b75-1600_jpg.rf.5d7117e8571505dbfe49e2f737089ea0.j